In [8]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


In [9]:
# poetry add docx2txt

In [10]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain import hub
from langchain.document_loaders import Docx2txtLoader
import warnings

warnings.filterwarnings("ignore", category=UserWarning)  # 특정 경고 유형만 무시

#  1. 환경 변수 로드
load_dotenv()

#  2. OpenAI API 키 확인
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OpenAI API 키가 설정되지 않았습니다. .env 파일을 확인하세요.")

#  3. DOCX 파일 로드 및 텍스트 추출 (Docx2txtLoader 활용)
def load_docx(file_path):
    """DOCX 파일에서 텍스트를 추출하는 함수."""
    try:
        loader = Docx2txtLoader(file_path)
        documents = loader.load()
        text = "\n".join([doc.page_content for doc in documents])
        if not text.strip():
            raise ValueError("문서에서 텍스트를 추출할 수 없습니다.")
        return text
    except Exception as e:
        raise RuntimeError(f"문서 로딩 실패: {str(e)}")

#  4. 문서 분할 함수
def split_text(text, chunk_size=500, chunk_overlap=50):
    """텍스트를 지정된 크기의 청크로 분할하는 함수."""
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    return splitter.split_text(text)

#  5. 벡터 데이터베이스(FAISS) 생성 함수
def create_vector_store(text_chunks, embedding_model):
    """텍스트 청크를 임베딩하고 FAISS 벡터 저장소에 저장."""
    try:
        documents = [Document(page_content=chunk) for chunk in text_chunks]
        vector_store = FAISS.from_documents(documents, embedding_model)
        return vector_store
    except Exception as e:
        raise RuntimeError(f"벡터 저장소 생성 실패: {str(e)}")

#  6. LLM을 활용한 질문 응답 함수
def query_with_llm(query, vector_store):
    """LLM을 사용하여 검색된 문서 기반으로 답변 생성."""
    try:
        # LLM 모델 설정
        llm = ChatOpenAI(model_name="gpt-3.5-turbo")
        
        # 프롬프트 로드 (RAG 최적화된 LangChain Hub 프롬프트 사용)
        #prompt = hub.pull("rlm/rag-prompt")
        prompt = hub.pull("cheorhyeon/rag-prompt-korean")
        print(prompt)

        # RetrievalQA 체인 생성
        qa_chain = RetrievalQA.from_chain_type(
            llm, 
            retriever=vector_store.as_retriever(),
            chain_type_kwargs={"prompt": prompt}
        )

        # LLM 응답 생성
        ai_message = qa_chain.invoke({"query": query})
        print(ai_message)
        return ai_message["result"]

    except Exception as e:
        raise RuntimeError(f"LLM 응답 생성 실패: {str(e)}")

In [12]:
#  실행 예제
if __name__ == "__main__":
    # DOCX 파일 경로
    docx_path = "data/tax_with_table_short.docx"
    
    # 1. 문서 로드
    text = load_docx(docx_path)
    print("문서 로드 완료")
    
    # 2. 문서 분할
    text_chunks = split_text(text)
    print(f"문서 분할 완료: {len(text_chunks)}개 청크 생성")
    
    # 3. 임베딩 모델 초기화
    embedding_model = OpenAIEmbeddings()
    
    # 4. 벡터 저장소 생성
    vector_store = create_vector_store(text_chunks, embedding_model)
    print("벡터 저장소 생성 완료")
    
    # 5. 질의 실행
    query = "총수입금액 불산입에 대하여 설명해 주세요."
    results = query_with_llm(query, vector_store)
    
    # 6. AI 응답 출력
    print("\n AI의 답변:")
    print(results)

문서 로드 완료
문서 분할 완료: 296개 청크 생성
벡터 저장소 생성 완료
input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'cheorhyeon', 'lc_hub_repo': 'rag-prompt-korean', 'lc_hub_commit_hash': '23289ae9d525acc8cd52f2e7be1b4a2b09f527f7fa5d42287cac0ab824a4b3ca'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="\n당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다.\n주어진 문맥(Context)에서 질문(Question)에 답하세요.\n답을 모르면 '주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다'라고 답하세요.\n모든 답변은 한글로 작성하되, 기술 용어나 이름은 번역하지 마세요.\n"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n#Question: {question}\n\n#Context: {context}\n'), additional_kwargs={})]
{'query': '총수입금액 불산입에 대하여 설명해 주세요.', 'result': '총수입금액 불산입이란 거주자가 자신의 소득금액을 계산할 때 일정한 조건에 해당하는 세금 환급액이나 일부 부가세 등을 총수입금액에 더하지 않는 것을 말합니다. 즉, 일부 법률로 인해 환급받은 세금 등은 총수입금액